# Evaluation of EfficientNet on CIFAR-10 (corrupted)



In [6]:
# Install required package
#!pip install timm --quiet

In [7]:
# Import packages

import timm
import os
import numpy as np
from torchvision import datasets, transforms, models
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import timm
from PIL import Image

In [8]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

#define transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

Using device: cuda


## Load and preprocess CIFAR-10C

In [9]:
CIFAR10C_PATH = "./cifar-10-c"  # download & extract from https://zenodo.org/record/2535967
corruptions = [
    'gaussian_noise', 'shot_noise', 'impulse_noise', 'defocus_blur', 'glass_blur',
    'motion_blur', 'zoom_blur', 'snow', 'frost', 'fog', 'brightness', 'contrast',
    'elastic_transform', 'pixelate', 'jpeg_compression'
]

# ==== Custom Dataset ====
from torch.utils.data import Dataset, DataLoader
class CIFAR10CDataset(Dataset):
    def __init__(self, data_np, labels_np, transform=None):
        self.data = data_np
        self.labels = labels_np
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = Image.fromarray(self.data[idx])
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# ==== Load Pretrained EfficientNet Model ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model architecture
model = models.efficientnet_v2_s(weights=None)  # Don't load pretrained weights here
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
model = model.to(device)
model.load_state_dict(torch.load("efficientnetv2_best.pth", map_location=device))
model.eval()


C:\Users\Ernest\AppData\Local\Temp\ipykernel_18216\389053069.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("efficientnetv2_best.pth",

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

## Load trained models

In [10]:
def evaluate_corruption(data_np, labels_np):
    dataset = CIFAR10CDataset(data_np, labels_np, transform=transform)
    loader = DataLoader(dataset, batch_size=64, shuffle=False, pin_memory=True)

    correct, total = 0, 0
    with torch.no_grad():
        for imgs, lbls in loader:
            imgs = imgs.to(device)
            lbls = lbls.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1)
            correct += (preds == lbls).sum().item()
            total += lbls.size(0)
    return 100 * correct / total

# ==== Load Common Labels (Same for All) ====
labels = np.load(os.path.join(CIFAR10C_PATH, "labels.npy"))


## load labels and define eval function

In [11]:
# ==== Evaluate All Corruptions ====
results = {}
for name in corruptions:
    print(f"Evaluating: {name}")
    data_np = np.load(os.path.join(CIFAR10C_PATH, f"{name}.npy"))
    acc = evaluate_corruption(data_np, labels)
    results[name] = acc
    print(f"{name}: {acc:.2f}%")

# ==== Summary ====
avg_acc = np.mean(list(results.values()))
print("\n===== CIFAR-10-C Evaluation Complete =====")
for k, v in results.items():
    print(f"{k:<20}: {v:.2f}%")
print(f"\nAverage Corruption Accuracy: {avg_acc:.2f}%")

Evaluating: gaussian_noise
gaussian_noise: 65.45%
Evaluating: shot_noise
shot_noise: 74.44%
Evaluating: impulse_noise
impulse_noise: 76.70%
Evaluating: defocus_blur
defocus_blur: 90.64%
Evaluating: glass_blur
glass_blur: 66.41%
Evaluating: motion_blur
motion_blur: 85.89%
Evaluating: zoom_blur
zoom_blur: 88.71%
Evaluating: snow
snow: 90.41%
Evaluating: frost
frost: 91.23%
Evaluating: fog
fog: 92.23%
Evaluating: brightness
brightness: 95.20%
Evaluating: contrast
contrast: 88.57%
Evaluating: elastic_transform
elastic_transform: 86.87%
Evaluating: pixelate
pixelate: 81.02%
Evaluating: jpeg_compression
jpeg_compression: 82.21%

===== CIFAR-10-C Evaluation Complete =====
gaussian_noise      : 65.45%
shot_noise          : 74.44%
impulse_noise       : 76.70%
defocus_blur        : 90.64%
glass_blur          : 66.41%
motion_blur         : 85.89%
zoom_blur           : 88.71%
snow                : 90.41%
frost               : 91.23%
fog                 : 92.23%
brightness          : 95.20%
contras